In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import xlrd

## Loading Data

#### To use this method, two files have to be saved in this direction: 

##### 1) Question Raw Data, named YYYYData.xls.  The "Report" sheet at the end MUST BE DELETED
##### 2) All Students, named YYYYStudents.xlsx.

In [141]:
def load_data(year):
    directory = './'+str(year)+'Data.xls'
    
    
    overallDF = pd.DataFrame()
    book = xlrd.open_workbook(directory) 
    num_sheets = len(book.sheets())

    sheet_names = book.sheet_names()
    sheet_names

    #GET ALL STUDENTS TO USE AS THE INDEX
    book_students = xlrd.open_workbook('./'+ str(year) + 'Students.xlsx')
    all_students = pd.read_excel('./'+ str(year) + 'Students.xlsx', 'Sheet1', header=None).iloc[:,0].tolist()
    
    possiblePoints = pd.Series() #creating a separate DF for possible points
    
    for i in range(num_sheets):
        this_sheet = sheet_names[i]
        df = pd.read_excel(directory, this_sheet)
        df.set_index(df.columns[0], inplace=True)
        
        possible_points_case = df.iloc[0]
        if (i == 0):
            print(type(possible_points_case))
        possiblePoints = possiblePoints.append(possible_points_case)
        
        df = df.reindex(all_students)
        df = df.reset_index()
        overallDF = pd.concat([overallDF, df], axis=1)
        
        
    

    return overallDF, possiblePoints

In [142]:
overalldf_2009, possible_points_2009 = load_data(2009)

<class 'pandas.core.series.Series'>


In [143]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
# print(len(overalldf_2009.select_dtypes(include=numerics).columns))
# print(len(overalldf_2009.columns))


overalldf_2009.head()

,Student Name_SP_TW,School ID_SP_TW,Class Year_SP_TW,Patient Name_SP_TW,Evaluator Name_SP_TW,Exam Date/Time_SP_TW,Q1_A_SP_TW,Q2_A_SP_TW,Q3_A_SP_TW,Q4_A_SP_TW,...,Q8_Hx_SP_LAP,Q9_Hx_SP_LAP,Q10_Hx_SP_LAP,Q11_Hx_SP_LAP,Q12_Hx_SP_LAP,Q13_Hx_SP_LAP,Q14_Hx_SP_LAP,Q15_Hx_SP_LAP,Total_Hx_SP_LAP,Total_SP_LAP
0,"Abbey, Ashkan",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Angarita, Benjamin",NaN,2009.0,"Levine, Neil","Levine, Neil",2008-08-12 11:00:04.420000,3,3,3,2,...,4,4,4,4,4,4,4,0,56.0,56.0
2,"Aniukwu, Jideofor",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Austin, Naola",NaN,2009.0,"Levine, Neil","Levine, Neil",2008-08-11 12:22:50.763000,4,4,3,3,...,0,4,0,4,4,4,4,4,48.0,48.0
4,"Ayodele, Maranatha",NaN,2009.0,"Levine, Neil","Levine, Neil",2008-07-22 13:26:49.560000,3,3,4,3,...,4,0,0,0,0,0,4,0,32.0,32.0


In [148]:
smino = ["Q1_PPI_SP_TW", "Q2_PPI_SP_TW", "Q3_PPI_SP_TW", "Q4_PPI_SP_TW"]

selected_cols = list(overalldf_2009.loc[:, overalldf_2009.columns.str.contains(('|'.join(smino)))].columns)

selected_points = possible_points_2009[["Q1_PPI_SP_TW", "Q2_PPI_SP_TW", "Q3_PPI_SP_TW", "Q4_PPI_SP_TW"]]

overalldf_2009[selected_cols].div(selected_points).mean(axis=1)

# list(overalldf_2009.loc[:, overalldf_2009.columns.str.contains(('|'.join(smino)))].columns)

0       NaN
1      1.00
2       NaN
3      1.00
4      1.00
5       NaN
6      1.00
7      1.00
8      1.00
9       NaN
10      NaN
11     1.00
12     1.00
13     1.00
14     1.00
15     1.00
16      NaN
17      NaN
18     0.75
19     1.00
20     1.00
21     1.00
22     1.00
23      NaN
24      NaN
25     1.00
26      NaN
27     1.00
28     1.00
29     1.00
       ... 
80     1.00
81      NaN
82     1.00
83      NaN
84     1.00
85     1.00
86     1.00
87     1.00
88     1.00
89     1.00
90      NaN
91      NaN
92     0.75
93      NaN
94      NaN
95      NaN
96      NaN
97     1.00
98     1.00
99      NaN
100    1.00
101     NaN
102    1.00
103     NaN
104    1.00
105    1.00
106    1.00
107     NaN
108     NaN
109    1.00
dtype: float64

In [4]:
# overalldf_2008 = load_data(2008)
# overalldf_2009 = load_data(2009)
# overalldf_2010 = load_data(2010)

In [5]:
# overalldf_2011 = load_data(2011)

## Get Mean and Variance

#### Identify pattern of columns for each subdivision.  Then find row-wise mean and variance for those selected columns and add to end of DF

In [6]:
# categories = ['ppi_initiation', 'ppi_info_gather', 'ppi_closing', 'hx_physical', 'hx_social', 'pe_handwash', 'pe_phys_check', 'pe_modesty', 'ps_personal', 'ps_rec']

# def generate_cols(categories):
#     col_names = []
#     for cat in categories:
#         col_names.append("mean_" + cat)
#         col_names.append("var_" + cat)
#     return col_names

# column_names = generate_cols(categories)

# combinedDF = pd.DataFrame(columns=column_names)

In [7]:
# combinedDF

,mean_ppi_initiation,var_ppi_initiation,mean_ppi_info_gather,var_ppi_info_gather,mean_ppi_closing,var_ppi_closing,mean_hx_physical,var_hx_physical,mean_hx_social,var_hx_social,mean_pe_handwash,var_pe_handwash,mean_pe_phys_check,var_pe_phys_check,mean_pe_modesty,var_pe_modesty,mean_ps_personal,var_ps_personal,mean_ps_rec,var_ps_rec


In [13]:
overalldf_2009.columns.tolist()

[u'Student Name_SP_TW',
 u'School ID_SP_TW',
 u'Class Year_SP_TW',
 u'Patient Name_SP_TW',
 u'Evaluator Name_SP_TW',
 u'Exam Date/Time_SP_TW',
 u'Q1_A_SP_TW',
 u'Q2_A_SP_TW',
 u'Q3_A_SP_TW',
 u'Q4_A_SP_TW',
 u'Q5_A_SP_TW',
 u'Q6_A_SP_TW',
 u'Q7_A_SP_TW',
 u'Q8_A_SP_TW',
 u'Q9_A_SP_TW',
 u'Q10_A_SP_TW',
 u'Total_A_SP_TW',
 u'Total_SP_TW',
 u'Student Name_SP_TW',
 u'School ID_SP_TW',
 u'Class Year_SP_TW',
 u'Patient Name_SP_TW',
 u'Evaluator Name_SP_TW',
 u'Exam Date/Time_SP_TW',
 u'Q1_PS_SP_TW',
 u'Q2_PS_SP_TW',
 u'Total_PS_SP_TW',
 u'Total_SP_TW',
 u'Student Name_SP_TW',
 u'School ID_SP_TW',
 u'Class Year_SP_TW',
 u'Patient Name_SP_TW',
 u'Evaluator Name_SP_TW',
 u'Exam Date/Time_SP_TW',
 u'Q1_PPI_SP_TW',
 u'Q2_PPI_SP_TW',
 u'Q3_PPI_SP_TW',
 u'Q4_PPI_SP_TW',
 u'Q5_PPI_SP_TW',
 u'Q6_PPI_SP_TW',
 u'Q7_PPI_SP_TW',
 u'Q8_PPI_SP_TW',
 u'Q9_PPI_SP_TW',
 u'Q10_PPI_SP_TW',
 u'Q11_PPI_SP_TW',
 u'Q12_PPI_SP_TW',
 u'Q13_PPI_SP_TW',
 u'Q14_PPI_SP_TW',
 u'Q15_PPI_SP_TW',
 u'Q16_PPI_SP_TW',
 u'Total

In [ ]:
# dict_2008 = {
#     'ppi_initiation': ["Q1_PPI", "Q2_PPI", "Q3_PPI", "Q4_PPI"], 
#     'ppi_info_gather': ["Q5_PPI", "Q6_PPI", "Q7_PPI", "Q8_PPI", "Q9_PPI", "Q10_PPI", "Q11_PPI"], 
#     'ppi_closing': ["Q12_PPI", "Q13_PPI", "Q14_PPI"], 
#     'hx_physical': ["Q1_Hx", "Q2_Hx", "Q3_Hx", "Q4_Hx", "Q5_Hx", "Q6_Hx", "Q7_Hx", "Q8_Hx", "Q9_Hx", "Q10_Hx", "Q11_Hx", "Q12_Hx"], 
#     'hx_social': ["Q13_Hx"], 
#     'pe_handwash': ["Q1_PE"], 
#     'pe_phys_check': ["Q2_PE", "Q4_PE", "Q5_PE", "Q6_PE", "Q7_PE", "Q8_PE", "Q9_PE", "Q10_PE", "Q11_PE", "Q12_PE", "Q13_PE", "Q14_PE", "Q15_PE", "Q16_PE", "Q17_PE", "Q18_PE", "Q19_PE", "Q20_PE"], 
#     'pe_modesty': ["Q3_PE"], 
#     'ps_personal': ['Q1_PS'], 
#     'ps_rec': ['Q2_PS']
# }

[9]

### 2009

In [90]:
overalldf_2009

,Student Name_SP_TW,School ID_SP_TW,Class Year_SP_TW,Patient Name_SP_TW,Evaluator Name_SP_TW,Exam Date/Time_SP_TW,Q1_A_SP_TW,Q2_A_SP_TW,Q3_A_SP_TW,Q4_A_SP_TW,...,Q8_Hx_SP_LAP,Q9_Hx_SP_LAP,Q10_Hx_SP_LAP,Q11_Hx_SP_LAP,Q12_Hx_SP_LAP,Q13_Hx_SP_LAP,Q14_Hx_SP_LAP,Q15_Hx_SP_LAP,Total_Hx_SP_LAP,Total_SP_LAP
0,"Abbey, Ashkan",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Angarita, Benjamin",NaN,2009.0,"Levine, Neil","Levine, Neil",2008-08-12 11:00:04.420000,3,3,3,2,...,4,4,4,4,4,4,4,0,56.0,56.0
2,"Aniukwu, Jideofor",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Austin, Naola",NaN,2009.0,"Levine, Neil","Levine, Neil",2008-08-11 12:22:50.763000,4,4,3,3,...,0,4,0,4,4,4,4,4,48.0,48.0
4,"Ayodele, Maranatha",NaN,2009.0,"Levine, Neil","Levine, Neil",2008-07-22 13:26:49.560000,3,3,4,3,...,4,0,0,0,0,0,4,0,32.0,32.0
5,"Barnhard, Sarah",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"Batavia, Ashita",NaN,2009.0,"Levine, Neil","Levine, Neil",2008-07-16 09:56:40.403000,3,3,3,3,...,0,4,0,0,0,0,4,0,36.0,36.0
7,"Beesley, Sarah",NaN,2009.0,"Levine, Neil","Levine, Neil",2008-07-24 12:06:31.140000,3,4,3,2,...,4,4,0,0,0,0,4,0,36.0,36.0
8,"Bergman, Meredith",NaN,2009.0,"Levine, Neil","Levine, Neil",2008-07-21 09:02:14.153000,3,2,3,2,...,4,4,0,0,0,4,4,4,44.0,44.0
9,"Beyene, Robel",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# cases_2009 = {
#     'AP': {
#         'ppi_initiation': ["Q1_PPI", "Q2_PPI", "Q3_PPI", "Q4_PPI"], 
#         'ppi_info_gather': ["Q5_PPI", "Q6_PPI", "Q7_PPI", "Q8_PPI", "Q9_PPI", "Q10_PPI", "Q11_PPI", "Q12_PPI"], 
#         'ppi_closing': ["Q13_PPI", "Q14_PPI", "Q15_PPI", "Q16_PPI"], 
#         'hx_physical': ["Q1_Hx", "Q2_Hx", "Q3_Hx", "Q4_Hx", "Q5_Hx", "Q6_Hx", "Q7_Hx", "Q8_Hx"], 
#         'hx_social': ["Q9_Hx"], 
#         'pe_handwash': ["Q1_PE"], 
#         'pe_phys_check': ["Q2_PE", "Q4_PE", "Q5_PE", "Q6_PE", "Q7_PE", "Q8_PE", "Q9_PE"], 
#         'pe_modesty': ["Q3_PE"], 
#         'ps_personal': ['Q1_PS'], 
#         'ps_rec': ['Q2_PS']
#     },
#     'TW': {
#         'ppi_initiation': ["Q1_PPI", "Q2_PPI", "Q3_PPI", "Q4_PPI"], 
#         'ppi_info_gather': ["Q5_PPI", "Q6_PPI", "Q7_PPI", "Q8_PPI", "Q9_PPI", "Q10_PPI", "Q11_PPI", "Q12_PPI"], 
#         'ppi_closing': ["Q13_PPI", "Q14_PPI", "Q15_PPI", "Q16_PPI"], 
#         'hx_physical': ["Q1_Hx", "Q2_Hx", "Q3_Hx", "Q4_Hx", "Q5_Hx", "Q6_Hx", "Q7_Hx", "Q8_Hx", "Q9_Hx", "Q10_Hx", "Q11_Hx", "Q12_Hx"], 
#         'hx_social': ["Q13_Hx"], 
#         'pe_handwash': ["Q1_PE"], 
#         'pe_phys_check': ["Q2_PE", "Q4_PE", "Q5_PE", "Q6_PE", "Q7_PE", "Q8_PE", "Q9_PE", "Q10_PE", "Q11_PE", "Q12_PE", "Q13_PE", "Q14_PE", "Q15_PE", "Q16_PE", "Q17_PE", "Q18_PE", "Q19_PE", "Q20_PE"], 
#         'pe_modesty': ["Q3_PE"], 
#         'ps_personal': ['Q1_PS'], 
#         'ps_rec': ['Q2_PS']
#     }
    
# }

In [47]:
# dict_2009 = {
#     'ppi_initiation': ["Q1_PPI", "Q2_PPI", "Q3_PPI", "Q4_PPI"], 
#     'ppi_info_gather': ["Q5_PPI", "Q6_PPI", "Q7_PPI", "Q8_PPI", "Q9_PPI", "Q10_PPI", "Q11_PPI", "Q12_PPI"], 
#     'ppi_closing': ["Q13_PPI", "Q14_PPI", "Q15_PPI", "Q16_PPI"], 
#     'hx_physical': ["Q1_Hx", "Q2_Hx", "Q3_Hx", "Q4_Hx", "Q5_Hx", "Q6_Hx", "Q7_Hx", "Q8_Hx"], 
#     'hx_social': ["Q9_Hx"], 
#     'pe_handwash': ["Q1_PE"], 
#     'pe_phys_check': ["Q2_PE", "Q4_PE", "Q5_PE", "Q6_PE", "Q7_PE", "Q8_PE", "Q9_PE"], 
#     'pe_modesty': ["Q3_PE"], 
#     'ps_personal': ['Q1_PS'], 
#     'ps_rec': ['Q2_PS']
# }

def generate_question_strings(first_q, last_q):
    def question_string(num):
        return "Q" + str(num)
    int_list = list(range(first_q,last_q+1))
    return [question_string(num) for num in int_list]

dict_2009 = {
    'PPI': {
        'init': {
            'AP': generate_question_strings(1,4),
            'TW': generate_question_strings(1,4)
        },
        'info_gather': {
            'AP': generate_question_strings(5,12),
            'TW': generate_question_strings(5,12)
        },
        'closing': {
            'AP': generate_question_strings(13,16),
            'TW': generate_question_strings(13,16)
        }
    },
    'Hx': {
        'physical': {
            'AP': generate_question_strings(1,8),
            'TW': generate_question_strings(1,12)
        },
        'social': {
            'AP': generate_question_strings(9,9),
            'TW': generate_question_strings(13,13)
        }

    },
    'PE': {
        'handwash': {
            'AP': generate_question_strings(1,1),
            'TW': generate_question_strings(1,1)
        },
        'phys_check': {
            'AP': generate_question_strings(2,2) + generate_question_strings(4,9),
            'TW': generate_question_strings(2,2) + generate_question_strings(4,20)
        },
        'modesty': {
            'AP': generate_question_strings(3,3),
            'TW': generate_question_strings(3,3)
        }
    },
    'PS': {
        'personal': {
            'AP': generate_question_strings(1,1),
            'TW': generate_question_strings(1,1)
        },
        'rec': {
            'AP': generate_question_strings(2,2),
            'TW': generate_question_strings(2,2)
        }
    }
}

# cases_2009 = {
#     'AP': {
#         'ppi_initiation': ["Q1_PPI", "Q2_PPI", "Q3_PPI", "Q4_PPI"], 
#         'ppi_info_gather': ["Q5_PPI", "Q6_PPI", "Q7_PPI", "Q8_PPI", "Q9_PPI", "Q10_PPI", "Q11_PPI", "Q12_PPI"], 
#         'ppi_closing': ["Q13_PPI", "Q14_PPI", "Q15_PPI", "Q16_PPI"], 
#         'hx_physical': ["Q1_Hx", "Q2_Hx", "Q3_Hx", "Q4_Hx", "Q5_Hx", "Q6_Hx", "Q7_Hx", "Q8_Hx"], 
#         'hx_social': ["Q9_Hx"], 
#         'pe_handwash': ["Q1_PE"], 
#         'pe_phys_check': ["Q2_PE", "Q4_PE", "Q5_PE", "Q6_PE", "Q7_PE", "Q8_PE", "Q9_PE"], 
#         'pe_modesty': ["Q3_PE"], 
#         'ps_personal': ['Q1_PS'], 
#         'ps_rec': ['Q2_PS']
#     },
#     'TW': {
#         'ppi_initiation': ["Q1_PPI", "Q2_PPI", "Q3_PPI", "Q4_PPI"], 
#         'ppi_info_gather': ["Q5_PPI", "Q6_PPI", "Q7_PPI", "Q8_PPI", "Q9_PPI", "Q10_PPI", "Q11_PPI", "Q12_PPI"], 
#         'ppi_closing': ["Q13_PPI", "Q14_PPI", "Q15_PPI", "Q16_PPI"], 
#         'hx_physical': ["Q1_Hx", "Q2_Hx", "Q3_Hx", "Q4_Hx", "Q5_Hx", "Q6_Hx", "Q7_Hx", "Q8_Hx", "Q9_Hx", "Q10_Hx", "Q11_Hx", "Q12_Hx"], 
#         'hx_social': ["Q13_Hx"], 
#         'pe_handwash': ["Q1_PE"], 
#         'pe_phys_check': ["Q2_PE", "Q4_PE", "Q5_PE", "Q6_PE", "Q7_PE", "Q8_PE", "Q9_PE", "Q10_PE", "Q11_PE", "Q12_PE", "Q13_PE", "Q14_PE", "Q15_PE", "Q16_PE", "Q17_PE", "Q18_PE", "Q19_PE", "Q20_PE"], 
#         'pe_modesty': ["Q3_PE"], 
#         'ps_personal': ['Q1_PS'], 
#         'ps_rec': ['Q2_PS']
#     }
    
# }

dict_2009



{'Hx': {'physical': {'AP': ['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8'],
   'TW': ['Q1',
    'Q2',
    'Q3',
    'Q4',
    'Q5',
    'Q6',
    'Q7',
    'Q8',
    'Q9',
    'Q10',
    'Q11',
    'Q12']},
  'social': {'AP': ['Q9'], 'TW': ['Q13']}},
 'PE': {'handwash': {'AP': ['Q1'], 'TW': ['Q1']},
  'modesty': {'AP': ['Q3'], 'TW': ['Q3']},
  'phys_check': {'AP': ['Q2', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9'],
   'TW': ['Q2',
    'Q4',
    'Q5',
    'Q6',
    'Q7',
    'Q8',
    'Q9',
    'Q10',
    'Q11',
    'Q12',
    'Q13',
    'Q14',
    'Q15',
    'Q16',
    'Q17',
    'Q18',
    'Q19',
    'Q20']}},
 'PPI': {'closing': {'AP': ['Q13', 'Q14', 'Q15', 'Q16'],
   'TW': ['Q13', 'Q14', 'Q15', 'Q16']},
  'info_gather': {'AP': ['Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12'],
   'TW': ['Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12']},
  'init': {'AP': ['Q1', 'Q2', 'Q3', 'Q4'], 'TW': ['Q1', 'Q2', 'Q3', 'Q4']}},
 'PS': {'personal': {'AP': ['Q1'], 'TW': ['Q1']},
  'rec': {'AP': ['Q2'], 

In [149]:
def get_year_df_new(yearDF, year_dict, possible_points):
    def construct_full_q(q, form_name, case_name):
        return q + "_" + form_name + "_SP_" + case_name
    
    new_year_df = pd.DataFrame()
    new_year_df["StudentName"] = yearDF.iloc[:,0]
    
    for form_name, sub_cat_dict in year_dict.iteritems():
        for sub_cat, case_dict in sub_cat_dict.iteritems():
            for case, questions in case_dict.iteritems():
                col_names = [construct_full_q(q, form_name, case) for q in questions]
                selected_cols = list(yearDF.loc[:, yearDF.columns.str.contains(('|'.join(col_names)))].columns)
                selected_points = possible_points[col_names]
                new_year_df["mean_" + form_name + sub_cat] = yearDF[selected_cols].div(selected_points).mean(axis=1)
                new_year_df["var_" + form_name + sub_cat] = yearDF[selected_cols].div(selected_points).var(axis=1)
    
    return new_year_df

In [9]:
# def get_year_df(yearDF, year_dict):
#     new_year_df = pd.DataFrame()
#     new_year_df["StudentName"] = yearDF.iloc[:,0]
#     for key, value in year_dict.iteritems():
#         selected_cols = list(yearDF.loc[:, yearDF.columns.str.contains(('|'.join(value)))].columns)
#         new_year_df["mean_" + key] = yearDF[selected_cols].mean(axis=1)
#         new_year_df["var_" + key] = yearDF[selected_cols].var(axis=1)
    
#     return new_year_df
    
# combinedDF = pd.concat([combinedDF, get_year_df(overalldf_2009, dict_2009)])


In [150]:
get_year_df_new(overalldf_2009, dict_2009, possible_points_2009)

,StudentName,mean_PSpersonal,var_PSpersonal,mean_PSrec,var_PSrec,mean_PPIinfo_gather,var_PPIinfo_gather,mean_PPIinit,var_PPIinit,mean_PPIclosing,...,mean_Hxsocial,var_Hxsocial,mean_Hxphysical,var_Hxphysical,mean_PEmodesty,var_PEmodesty,mean_PEhandwash,var_PEhandwash,mean_PEphys_check,var_PEphys_check
0,"Abbey, Ashkan",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Angarita, Benjamin",0.75,NaN,0.25,NaN,0.875,0.125000,1.00,0.00,1.00,...,1.0,NaN,0.666667,0.242424,1.0,NaN,0.0,NaN,0.222222,0.183007
2,"Aniukwu, Jideofor",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Austin, Naola",1.00,NaN,1.00,NaN,1.000,0.000000,1.00,0.00,1.00,...,0.0,NaN,0.750000,0.204545,1.0,NaN,1.0,NaN,0.444444,0.261438
4,"Ayodele, Maranatha",1.00,NaN,1.00,NaN,1.000,0.000000,1.00,0.00,1.00,...,1.0,NaN,0.833333,0.151515,1.0,NaN,1.0,NaN,0.500000,0.264706
5,"Barnhard, Sarah",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"Batavia, Ashita",1.00,NaN,1.00,NaN,1.000,0.000000,1.00,0.00,1.00,...,1.0,NaN,0.666667,0.242424,1.0,NaN,1.0,NaN,0.277778,0.212418
7,"Beesley, Sarah",1.00,NaN,0.75,NaN,1.000,0.000000,1.00,0.00,1.00,...,1.0,NaN,0.833333,0.151515,1.0,NaN,1.0,NaN,0.722222,0.212418
8,"Bergman, Meredith",0.75,NaN,0.75,NaN,0.875,0.125000,1.00,0.00,0.50,...,1.0,NaN,0.666667,0.242424,1.0,NaN,1.0,NaN,0.444444,0.261438
9,"Beyene, Robel",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


TypeError: cannot concatenate 'str' and 'dict' objects

In [10]:
combinedDF.head()

,StudentName,mean_hx_physical,mean_hx_social,mean_pe_handwash,mean_pe_modesty,mean_pe_phys_check,mean_ppi_closing,mean_ppi_info_gather,mean_ppi_initiation,mean_ps_personal,...,var_hx_physical,var_hx_social,var_pe_handwash,var_pe_modesty,var_pe_phys_check,var_ppi_closing,var_ppi_info_gather,var_ppi_initiation,var_ps_personal,var_ps_rec
0,"Abbey, Ashkan",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Angarita, Benjamin",2.871795,3.555556,1.333333,2.666667,1.706842,2.266667,2.807692,2.769231,1.0,...,3.282051,1.777778,4.000000,4.000000,3.901740,4.064368,3.380134,3.475113,1.111111,0.488889
2,"Aniukwu, Jideofor",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Austin, Naola",2.974359,1.333333,2.666667,3.555556,2.259649,3.333333,3.692308,3.923077,2.9,...,3.090243,4.000000,4.000000,1.777778,3.951378,2.298851,1.147125,0.307692,0.766667,0.988889
4,"Ayodele, Maranatha",2.820513,3.111111,3.555556,3.111111,2.282281,2.933333,3.692308,3.692308,2.3,...,3.369963,3.111111,1.777778,3.111111,3.856682,3.236782,1.147125,1.158371,1.344444,1.555556


## CLEAN BELOW THIS LINE

In [11]:
def add_mean_and_var_column(question_names, new_col_name):
    selected_cols = list(overallDF.loc[:, overallDF.columns.str.contains(('|'.join(question_names)))].columns)
    overallDF["mean_" + new_col_name] = overallDF[selected_cols].mean(axis=1)
    overallDF["var_" + new_col_name] = overallDF[selected_cols].var(axis=1)

add_mean_and_var_column(ppi_initiation_qs, 'ppi_initiation')
add_mean_and_var_column(ppi_info_gather, 'ppi_info_gather')
add_mean_and_var_column(ppi_closing, 'ppi_closing')

NameError: name 'ppi_initiation_qs' is not defined

In [ ]:
overallDF.columns.tolist()

In [ ]:
hx_physical =["Q1_Hx", "Q2_Hx", "Q3_Hx", "Q4_Hx", "Q5_Hx", "Q6_Hx", "Q7_Hx", "Q8_Hx"]
hx_social = ["Q9_Hx"]

add_mean_and_var_column(hx_physical, 'hx_physical')
add_mean_and_var_column(hx_social, 'hx_social')

In [ ]:
pe_handwash = ["Q1_PE"]
pe_phys_check = ["Q2_PE", "Q4_PE", "Q5_PE", "Q6_PE", "Q7_PE", "Q8_PE", "Q9_PE"]
pe_modesty = ["Q3_PE"]

add_mean_and_var_column(pe_handwash, 'pe_handwash')
add_mean_and_var_column(pe_phys_check, 'pe_phys_check')
add_mean_and_var_column(pe_modesty, 'pe_modesty')